In [0]:
%sql
CREATE OR REPLACE VIEW transacoes_db.gold.dimensao_cliente as 
select 
  clientes.id,
  clientes.nome,
  clientes.registro_nacional,
  clientes.nascido_em,
  clientes.idade,
  naturezas.id as id_natureza,
  naturezas.nome as natureza,
  enderecos.id_municipio,
  enderecos.bairro,
  enderecos.cep,
  enderecos.complemento,
  enderecos.logradouro,
  enderecos.numero 
from transacoes_db.silver.clientes clientes
JOIN transacoes_db.silver.naturezas naturezas ON naturezas.id = clientes.id_natureza
JOIN transacoes_db.silver.enderecos enderecos ON enderecos.id_cliente = clientes.id

In [0]:
CREATE OR REPLACE VIEW transacoes_db.gold.dimensao_contas as 
SELECT 
  contas.id,
  contas.saldo,
  contas.aberta_em,
  contas.agencia,
  contas.numero,
  contas.id_cliente,
  contas.ispb_instituicao,
  contas.tempo_conta_ativa_d,
  tipos_conta.nome as tipo_conta
from transacoes_db.silver.contas 
JOIN transacoes_db.silver.tipos_conta on tipos_conta.id = contas.id_tipo_conta

In [0]:
CREATE OR REPLACE VIEW transacoes_db.gold.dimensao_municipios as 
select 
  municipios.nome as municipio,
  municipios.id as id_municipio,
  municipios.id_estado,
  municipios.id_ibge municipio_ibge,
  estados.id_ibge AS estado_ibge,
  estados.nome as estado,
  estados.sigla as sigla_estado,
  regioes.nome as regiao
from transacoes_db.silver.municipios
JOIN transacoes_db.silver.estados on estados.id_ibge = municipios.id_estado
JOIN transacoes_db.silver.regioes on regioes.id = estados.id_regiao

In [0]:
CREATE OR REPLACE VIEW transacoes_db.gold.dimensao_chaves as 
select 
chaves_pix.id,
chaves_pix.id_conta,
tipos_chave_pix.nome as tipo_chave,
chaves_pix.chave,
chaves_pix.tempo_chave_ativa_d  
from transacoes_db.silver.chaves_pix
JOIN transacoes_db.silver.tipos_chave_pix on chaves_pix.id_tipo_chave = tipos_chave_pix.id

In [0]:
create or replace view transacoes_db.gold.fato_transacoes as
select 
  transacoes.id,
  transacoes.valor,
  transacoes.data,
  transacoes.mensagem,
  transacoes.id_conta_origem,
  transacoes.id_conta_destino,
  tipos_iniciacao_pix.nome as tipo_iniciacao,
  finalidade_pix.nome as finalidade
from transacoes_db.silver.transacoes
JOIN transacoes_db.silver.tipos_iniciacao_pix on tipos_iniciacao_pix.id = transacoes.id_finalidade_pix
JOIN transacoes_db.silver.finalidade_pix on finalidade_pix.id = transacoes.id_finalidade_pix


In [0]:
CREATE OR REPLACE VIEW transacoes_db.gold.transacoes_dataset AS
SELECT
  ft.id AS id_transacao,
  ft.valor,
  ft.data,
  ft.mensagem,
  ft.tipo_iniciacao,
  ft.finalidade,
  co.id AS id_conta_origem,
  co.saldo AS saldo_origem,
  co.aberta_em AS aberta_em_origem,
  co.agencia AS agencia_origem,
  co.numero AS numero_conta_origem,
  co.ispb_instituicao AS ispb_instituicao_origem,
  co.tempo_conta_ativa_d AS tempo_conta_ativa_origem,
  co.tipo_conta AS tipo_conta_origem,
  cd.id AS id_cliente_origem,
  cd.nome AS nome_cliente_origem,
  cd.registro_nacional AS registro_nacional_origem,
  cd.nascido_em AS nascido_em_origem,
  cd.idade AS idade_origem,
  cd.id_natureza AS id_natureza_origem,
  cd.natureza AS natureza_origem,
  cd.id_municipio AS id_municipio_origem,
  cd.bairro AS bairro_origem,
  cd.cep AS cep_origem,
  cd.complemento AS complemento_origem,
  cd.logradouro AS logradouro_origem,
  cd.numero AS numero_endereco_origem,
  mo.municipio AS municipio_origem,
  mo.estado AS estado_origem,
  mo.sigla_estado AS sigla_estado_origem,
  mo.regiao AS regiao_origem,
  cd2.id AS id_cliente_destino,
  cd2.nome AS nome_cliente_destino,
  cd2.registro_nacional AS registro_nacional_destino,
  cd2.nascido_em AS nascido_em_destino,
  cd2.idade AS idade_destino,
  cd2.id_natureza AS id_natureza_destino,
  cd2.natureza AS natureza_destino,
  cd2.id_municipio AS id_municipio_destino,
  cd2.bairro AS bairro_destino,
  cd2.cep AS cep_destino,
  cd2.complemento AS complemento_destino,
  cd2.logradouro AS logradouro_destino,
  cd2.numero AS numero_endereco_destino,
  md.municipio AS municipio_destino,
  md.estado AS estado_destino,
  md.sigla_estado AS sigla_estado_destino,
  md.regiao AS regiao_destino,
  ch.id AS id_chave_pix,
  ch.id_conta AS id_conta_chave_pix,
  ch.tipo_chave,
  ch.chave,
  ch.tempo_chave_ativa_d
FROM transacoes_db.gold.fato_transacoes ft
LEFT JOIN transacoes_db.gold.dimensao_contas co ON ft.id_conta_origem = co.id
LEFT JOIN transacoes_db.gold.dimensao_cliente cd ON co.id_cliente = cd.id
LEFT JOIN transacoes_db.gold.dimensao_municipios mo ON cd.id_municipio = mo.id_municipio
LEFT JOIN transacoes_db.gold.dimensao_contas cdest ON ft.id_conta_destino = cdest.id
LEFT JOIN transacoes_db.gold.dimensao_cliente cd2 ON cdest.id_cliente = cd2.id
LEFT JOIN transacoes_db.gold.dimensao_municipios md ON cd2.id_municipio = md.id_municipio
LEFT JOIN transacoes_db.gold.dimensao_chaves ch ON ch.id_conta = co.id